In [2]:
import numpy as np


In [17]:
def p_of_r(r, P_MAX, sigma, mu):
    """
    Compute the value of the function p(r) given the parameters P_MAX, sigma, and mu.

    :param r: The independent variable, r
    :param P_MAX: The maximum value P_MAX
    :param sigma: The standard deviation sigma
    :param mu: The mean value mu
    :return: The computed value of p(r)
    """
    return P_MAX * (1 - 1 / (1 + np.exp(-sigma * (r - mu))))

# # Assuming values for P_MAX, sigma, and mu
# P_MAX = 1  # This should be replaced with the actual value
# sigma = 1  # This should be replaced with the actual value
# mu = 0     # This should be replaced with the actual value
# r = 0.5    # This should be replaced with the value of r for which p(r) needs to be calculated

# # Calculate p(r)
# p_r = p_of_r(r, P_MAX, sigma, mu)
# p_r

In [18]:
def generate_points_and_distances_in_micrometers(n, size=0.001):
    """
    Generates n points in a square of specified size with labels 'E' or 'I' and computes the toroidal distance 
    between each pair of points. The square represents a 100 micrometer by 100 micrometer area.

    :param n: Number of points to generate
    :param size: Size of the square in micrometers
    :return: labels for the points and the matrix of distances in micrometers
    """
    # Generate random points in the square scaled by size
    points = np.random.rand(n, 2) * size
    
    # Assign labels 'E' or 'I' randomly to the points
    labels = np.random.choice(['E', 'I'], size=n)
    
    # Initialize the distance matrix
    NeuronDistance = np.zeros((n, n))
    
    # Compute the toroidal distance between each pair of points
    for i in range(n):
        for j in range(i+1, n):
            # Compute the minimum distance considering the torus topology
            dx = min(abs(points[i,0] - points[j,0]), size - abs(points[i,0] - points[j,0]))
            dy = min(abs(points[i,1] - points[j,1]), size - abs(points[i,1] - points[j,1]))
            # Calculate the Euclidean distance with the toroidal correction
            distance = np.sqrt(dx**2 + dy**2)
            NeuronDistance[i, j] = distance
            NeuronDistance[j, i] = distance  # Matrix is symmetric
    
    return labels, NeuronDistance

# Example usage for 5 points in a 100x100 micrometer square
labels, NeuronDistance = generate_points_and_distances_in_micrometers(5)
labels, NeuronDistance


(array(['E', 'E', 'E', 'I', 'E'], dtype='<U1'),
 array([[0.00000000e+00, 4.49035717e-04, 4.29494990e-04, 4.28561605e-04,
         4.54737974e-04],
        [4.49035717e-04, 0.00000000e+00, 2.25683031e-04, 5.59644660e-04,
         1.47802645e-04],
        [4.29494990e-04, 2.25683031e-04, 0.00000000e+00, 3.52199985e-04,
         8.73148224e-05],
        [4.28561605e-04, 5.59644660e-04, 3.52199985e-04, 0.00000000e+00,
         4.12834432e-04],
        [4.54737974e-04, 1.47802645e-04, 8.73148224e-05, 4.12834432e-04,
         0.00000000e+00]]))

In [19]:
def p_of_r_toroidal_with_types(NeuronDistance, labels):
    """
    Apply the probability function to the toroidal distance matrix with different constants for different types
    of neuron connections (EE, EI, and IE).

    :param NeuronDistance: Matrix of distances between neurons
    :param labels: Array of labels for each neuron
    :return: A matrix of probabilities
    """
    # Constants for the probability function for different neuron types
    P_MAX_EEI = 0.4
    sigma_EEI = 1.0
    mu_EEI = 10.0
    P_MAX_IE = 0.5
    sigma_IE = 1.0
    mu_IE = 10.0
    
    n = len(labels)
    ProbabilityMatrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i != j:  # No self-connection
                # Determine the type of connection (EE, EI, or IE)
                if labels[i] == 'E' and labels[j] == 'E':
                    P_MAX = P_MAX_EEI
                    sigma = sigma_EEI
                    mu = mu_EEI
                elif labels[i] == 'E' and labels[j] == 'I':
                    P_MAX = P_MAX_EEI
                    sigma = sigma_EEI
                    mu = mu_EEI
                elif labels[i] == 'I' and labels[j] == 'E':
                    P_MAX = P_MAX_IE
                    sigma = sigma_IE
                    mu = mu_IE
                
                r = NeuronDistance[i, j]
                ProbabilityMatrix[i, j] = p_of_r(r, P_MAX, sigma, mu)
    return ProbabilityMatrix

# Apply the probability function with type-specific constants
ProbabilityMatrixWithType = p_of_r_toroidal_with_types(NeuronDistance, labels)
ProbabilityMatrixWithType


array([[0.        , 0.39998183, 0.39998183, 0.39998183, 0.39998183],
       [0.39998183, 0.        , 0.39998184, 0.39998183, 0.39998184],
       [0.39998183, 0.39998184, 0.        , 0.39998183, 0.39998184],
       [0.49997729, 0.49997729, 0.49997729, 0.        , 0.49997729],
       [0.39998183, 0.39998184, 0.39998184, 0.39998183, 0.        ]])